# アダマールテストでの位数推定

本来ショアのアルゴリズムでは位相推定アルゴリズムを用いて固有値を高い精度で求め、それを用いて位数測定している。しかし今回は位相推定アルゴリズムを使う代わりにアダマールテストで固有値を推定しても位数推定ができると考え、実際に行列を用いたシュミレーターを作り実験して見ようと考えた。

In [3]:
import numpy as np
import math

In [4]:
def get_CU_gate(U):
    m,m=U.shape
    n=int(math.log(m,2))
    
    I=np.array([[1,0],[0,1]])
    CU_zero=np.array([[1,0],[0,0]])
    CU_one=np.array([[0,0],[0,1]])
    
    for i in range(n):
            CU_zero=np.kron(CU_zero,I)
    
    CU_one=np.kron(CU_one,U)
    
    return CU_zero+CU_one

In [5]:
def get_H_gate(U):
    m,m=U.shape
    n=int(math.log(m,2))
    
    H=np.array([[1,1],[1,-1]])/math.sqrt(2)
    I=np.array([[1,0],[0,1]])
    
    H_gate=H
    
    for i in range(n):
        H_gate=np.kron(H_gate,I)
    
    return H_gate

In [6]:
def hadamard_test(U,phi):
    
    # 初期状態を定める
    zero_bit=np.array([[1],[0]])
    state=np.kron(zero_bit,phi)
    
    # stateにH_gateを作用させる
    state=np.dot(get_H_gate(U),state)
    
    # stateにCU_gateを作用させる
    state=np.dot(get_CU_gate(U),state)
    
    # stateにH_gateを作用させる
    state=np.dot(get_H_gate(U),state)
    
    return state

In [7]:
def get_lambda(U,loop):
    m,m=U.shape
    n=int(math.log(m,2))
    
    state=np.zeros((2**n,1))
    state[1]=1 # 初期ビットは|1>で固定する
    
    for i in range(loop):
        end_state=hadamard_test(U,state)
        # アダマールテストで帰ってきたend_stateから１ビット目の情報を抜き取って、次のhadamard_testに使う(できれば再帰関数でかきたい)
        end_state0, end_state1=np.split(end_state,[2**n],axis=0) # end_stateを上下に分ける
        PZero=sum([abs(end_state[i,0]**2) for i in range(2**(n-1))]) # １ビット目で0が観測される確率
        if PZero==0:
            state=end_state1
        else:
            state=end_state0/((PZero)**(1/2))
            
    cosL=2*PZero-1
    cosL=min(cosL,1) # 1超えたときの保険

    return math.acos(cosL) # 返すのは固有値ではなくlambdaの値

In [8]:
def get_answer(U):
    w,v=np.linalg.eig(U)
    
    for i in range(len(w)):
        print('value:',w[i],' vector:',v[:,i]) 

## やりたいことの確認
1. 因数分解したい数Nに対して、Nと互いに素でありNより小さい自然数xを定める。  
2. そのNとxを使って、ユニタリ行列$U_{x}=\sum_{y=0}^{2^{n}-1} |xy (mod N)><y|$ を行列表現する。($2^{n-1}-1<N-1<=2^{n}-1$)  
3. その$U_{x}$の固有値$e^{i\lambda}$、というより$\lambda$をアダマールループで推定する。  
4. $s/t=\lambda/2\pi$に対して連分数展開を行い$t$を求める。

Nに対する適当なxと位数rを求める関数  
条件を満たす組み合わせが見つかると一度関数を中断して値を返すyieldが使われている  
for-elseのelseの処理はfor文終了時に処理されるが、breakで終わったときは実行しない

$U_{x}$を行列表現する関数

In [9]:
def get_Ux(N,x):
    # Nに対するnを求める
    i=0
    while(True):
        if N<=2**i:
            n=i
            break
        else:
            i+=1
    
    Ux=np.zeros((2**n,2**n))
    
    for y in range(2**n):
        ket=np.zeros((2**n,1))
        ket[(x*y)%N,0]=1
        bra=np.zeros((1,2**n))
        bra[0,y]=1
        Ux=Ux+np.kron(ket,bra)
    
    return Ux

In [10]:
get_Ux(4,3)

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

アダマールループで$\lambda$を推定する

In [11]:
Ux=get_Ux(12,5)
get_lambda(Ux,50) # Uxをアダマールテストを50回ループ

2.9802322387695312e-08

あ、これあらゆるUxに対して固有値1に対する$\lambda$しか求まってねぇ。いやこの行列が絶対に1を固有値に持つのか。どうやったって同じ位数が求まるじゃねえか、そんなんありかよ。やっぱアダマールループじゃ限界があるのか?いや初期ビット変えればいけるかも。無理でした。なんで0にいくねん。もっと固有的なもんに収束せんかい。もう限界が見えてきたけれどもせめて連分数だけでもやっとこう。

$\lambda$から連分数を求め、近似分数を求め、$t$を求める

有理数近似を取得するメソッドがあるみたい。一回使ってみよう。

In [12]:
from fractions import Fraction

pi = Fraction(3.14159265359)
print(pi)

print(pi.limit_denominator(10))
print(pi.limit_denominator(100))
print(pi.limit_denominator(1000))

3537118876014453/1125899906842624
22/7
311/99
355/113


いいねぇいいねぇ。時間も無いし使わせてもらいましょう。

In [19]:
Ux=get_Ux(12,5)
L=get_lambda(Ux,10)

print(L/2*math.pi)

st=Fraction(L/2*math.pi)
print(st)
print(st.limit_denominator(7550000))
print(st.limit_denominator(76000000))

4.681337853654911e-08
884279719003555/18889465931478580854784
0
1/21361415


やっぱアダマールループじゃだめか...

In [ ]:
get_answer(Ux)

でもこいつ0かプラマイ1にしかならんな。もしかして行列作り間違えたかなぁ。  
でも固有値-1の時はL=piか。pi/(2\*pi)=1/2で位数は2か。$5^{2}=1(mod 12)$だ。あってんじゃん。  
やっぱアダマールループでは思い通りの固有値にいかないからだめなのか。 
裏返せば位相推定アルゴリズムならちょうど良い固有値のLを取得できる?  
しょうが無いけど、アダマールループは一端中断。

もしかしたら固有状態に|1>を入れることで何か変わるかも。  
変わらないならそれはおそらくこの方法では無理

やっぱ無理した